In [1]:
from datasrcs.spc import load_tor_segments
from datasrcs.stormevents import load_tor_events

import pandas as pd
import numpy as np

%env WORKDIR=~/weatherpy-work

env: WORKDIR=~/weatherpy-work


In [5]:
spc_segs = load_tor_segments(to_tz='CST')
spc_03 = spc_segs[spc_segs.yr == 2003]

stevents_03 = load_tor_events('2003-01-01', '2003-12-31', tz='CST')

In [6]:
spc_03.columns

Index(['date_time', 'om', 'yr', 'mo', 'dy', 'tz', 'st', 'stf', 'stn', 'mag',
       'inj', 'fat', 'loss', 'closs', 'slat', 'slon', 'elat', 'elon', 'len',
       'wid', 'ns', 'sn', 'sg', 'f1', 'f2', 'f3', 'f4', 'fc'],
      dtype='object')

In [19]:
spc_cols = ['date_time', 'slat', 'slon', 'elat', 'elon', 'len', 'mag', 'f1', 'f2', 'om']
# spc_03[['slat1', 'slon1', 'elat1', 'elon1']] = tojoinable(spc_subset[['slat', 'slon', 'elat', 'elon']])
spc_subset = spc_03[spc_cols]
spc_subset.sort_values(by='date_time').head()

,date_time,slat,slon,elat,elon,len,mag,f1,f2,om
44410,2003-02-15 12:00:00,29.95,-92.00,29.95,-92.00,0.5,1,113,0,9
44411,2003-02-15 16:20:00,32.13,-89.78,32.15,-89.78,1.0,0,121,0,688
44412,2003-02-15 16:43:00,32.18,-89.60,32.27,-89.53,3.5,0,129,123,689
44413,2003-02-15 17:25:00,32.35,-89.27,32.37,-89.23,2.0,1,101,0,690
44414,2003-02-15 20:36:00,32.33,-87.82,32.37,-87.77,2.5,1,91,0,4


In [20]:
stevent_cols = ['begin_date_time', 'end_date_time', 'begin_lat', 'begin_lon', 'end_lat', 'end_lon', 'event_id']
# stevents_03[['begin_lat', 'begin_lon', 'end_lat', 'end_lon']] = stevents_03[['begin_lat', 'begin_lon', 'end_lat', 'end_lon']].round(2)
# stevents_03[['begin_lat1', 'begin_lon1', 'end_lat1', 'end_lon1']] = tojoinable(stevents_03[['begin_lat', 'begin_lon', 'end_lat', 'end_lon']])
stevents_subset = stevents_03[stevent_cols].round(2)
stevents_subset.sort_values(by='begin_date_time').head()

,begin_date_time,end_date_time,begin_lat,begin_lon,end_lat,end_lon,event_id
226,2003-02-15 12:00:00,2003-02-15 12:02:00,29.95,-92.00,29.95,-92.00,5344180
161,2003-02-15 16:20:00,2003-02-15 16:22:00,32.13,-89.78,32.15,-89.78,5342364
192,2003-02-15 16:43:00,2003-02-15 16:48:00,32.18,-89.60,32.28,-89.58,5342466
193,2003-02-15 16:48:00,2003-02-15 16:53:00,32.23,-89.55,32.25,-89.53,5342467
187,2003-02-15 17:25:00,2003-02-15 17:27:00,32.35,-89.27,32.37,-89.23,5342469


In [21]:
merged = spc_subset.merge(stevents_subset, 
             left_on=['date_time', 'slat', 'slon'], 
             right_on=['begin_date_time', 'begin_lat', 'begin_lon'],
             how='outer', indicator=True)

In [22]:
core = merged[merged._merge == 'both'] 

In [23]:
spconly = merged.loc[merged._merge == 'left_only', spc_cols]
steventsonly = merged.loc[merged._merge == 'right_only', stevent_cols]

In [24]:
spcwithf2 = spc_subset.loc[spc_subset.f2 != 0, spc_cols]
end_merged = steventsonly.merge(spcwithf2, left_on=['end_lat', 'end_lon'], 
                                right_on=['elat', 'elon'], indicator=True, how='outer')

In [25]:
segments = end_merged[end_merged._merge == 'both']
outliers_stevents = end_merged[end_merged._merge == 'left_only']
outliers_spc = end_merged[end_merged._merge == 'right_only']

In [26]:
len(core), len(segments), len(outliers_stevents)

(1294, 108, 152)

In [27]:
core.head()

,date_time,slat,slon,elat,elon,len,mag,f1,f2,om,begin_date_time,end_date_time,begin_lat,begin_lon,end_lat,end_lon,event_id,_merge
0,2003-02-15 12:00:00,29.95,-92.00,29.95,-92.00,0.5,1.0,113.0,0.0,9.0,2003-02-15 12:00:00,2003-02-15 12:02:00,29.95,-92.00,29.95,-92.00,5344180.0,both
1,2003-02-15 16:20:00,32.13,-89.78,32.15,-89.78,1.0,0.0,121.0,0.0,688.0,2003-02-15 16:20:00,2003-02-15 16:22:00,32.13,-89.78,32.15,-89.78,5342364.0,both
2,2003-02-15 16:43:00,32.18,-89.60,32.27,-89.53,3.5,0.0,129.0,123.0,689.0,2003-02-15 16:43:00,2003-02-15 16:48:00,32.18,-89.60,32.28,-89.58,5342466.0,both
3,2003-02-15 17:25:00,32.35,-89.27,32.37,-89.23,2.0,1.0,101.0,0.0,690.0,2003-02-15 17:25:00,2003-02-15 17:27:00,32.35,-89.27,32.37,-89.23,5342469.0,both
4,2003-02-15 20:36:00,32.33,-87.82,32.37,-87.77,2.5,1.0,91.0,0.0,4.0,2003-02-15 20:36:00,2003-02-15 20:40:00,32.33,-87.82,32.37,-87.77,5340389.0,both


In [28]:
segments.head()

,begin_date_time,end_date_time,begin_lat,begin_lon,end_lat,end_lon,event_id,date_time,slat,slon,elat,elon,len,mag,f1,f2,om,_merge
5,2003-07-13 13:25:00,2003-07-13 13:40:00,34.78,-82.58,34.73,-82.52,5329605.0,2003-07-13 13:15:00,34.78,-82.68,34.73,-82.52,12.0,1.0,77.0,7.0,565.0,both
7,2003-11-26 22:15:00,2003-11-26 22:20:00,30.90,-93.10,30.92,-93.08,5329059.0,2003-11-26 21:50:00,30.75,-93.32,30.92,-93.08,18.0,2.0,11.0,115.0,685.0,both
8,2003-07-13 13:30:00,2003-07-13 13:45:00,34.75,-82.63,34.75,-82.53,5329643.0,2003-07-13 13:20:00,34.80,-82.70,34.75,-82.53,12.0,1.0,77.0,7.0,566.0,both
9,2003-07-19 15:04:00,2003-07-19 15:05:00,45.98,-94.65,45.98,-94.65,5329610.0,2003-07-19 15:00:00,46.02,-94.72,45.98,-94.65,3.0,0.0,153.0,97.0,542.0,both
12,2003-11-17 12:20:00,2003-11-17 12:25:00,29.62,-95.55,29.62,-95.55,5331341.0,2003-11-17 12:15:00,29.63,-95.57,29.62,-95.55,1.0,2.0,157.0,201.0,1037.0,both


In [29]:
outliers_stevents[stevent_cols].sort_values('begin_date_time')

,begin_date_time,end_date_time,begin_lat,begin_lon,end_lat,end_lon,event_id
29,2003-02-15 16:48:00,2003-02-15 16:53:00,32.23,-89.55,32.25,-89.53,5342467.0
28,2003-03-27 13:20:00,2003-03-27 13:25:00,26.18,-814.08,26.17,-81.75,5344665.0
31,2003-03-29 17:25:00,2003-03-29 17:25:00,38.52,-770.02,38.52,-770.02,5347621.0
81,2003-04-04 15:11:00,2003-04-04 15:11:00,40.25,-89.23,40.25,-89.23,5350606.0
35,2003-04-04 15:48:00,2003-04-04 15:48:00,40.18,-88.97,40.18,-88.97,5350405.0
36,2003-04-04 15:53:00,2003-04-04 15:53:00,40.18,-88.82,40.18,-88.82,5350406.0
34,2003-04-04 16:08:00,2003-04-04 16:18:00,38.48,-89.47,38.53,-89.42,5350410.0
39,2003-04-04 18:35:00,2003-04-04 18:35:00,39.78,-87.88,39.78,-87.88,5350505.0
69,2003-04-05 15:00:00,2003-04-05 15:00:00,32.90,-100.40,32.90,-100.40,5352792.0
82,2003-04-06 10:57:00,2003-04-06 11:03:00,32.67,-90.38,32.67,-90.32,5350938.0


In [30]:
criteria = outliers_stevents.begin_date_time.between('2003-07-01', '2003-07-13')
stevent_july03 = outliers_stevents[stevent_cols][criteria]
july03 = stevent_july03.merge(spc_03[spc_cols], how='inner', left_on='begin_date_time', right_on='date_time')
july03

,begin_date_time,end_date_time,begin_lat,begin_lon,end_lat,end_lon,event_id,date_time,slat,slon,elat,elon,len,mag,f1,f2,om
0,2003-07-12 19:00:00,2003-07-12 19:25:00,35.37,-812.00,35.30,-81.02,5329529.0,2003-07-12 19:00:00,35.37,-81.35,35.30,-81.02,18.0,1,71,0,564
1,2003-07-09 17:22:00,2003-07-09 17:22:00,40.07,-88.30,40.07,-88.30,5329927.0,2003-07-09 17:22:00,40.08,-88.30,40.08,-88.30,0.1,0,19,0,561
2,2003-07-10 17:42:00,2003-07-10 17:55:00,38.25,-772.05,38.30,-77.33,5368413.0,2003-07-10 17:42:00,38.27,-77.40,38.30,-77.33,5.0,0,179,0,146
3,2003-07-02 12:40:00,2003-07-02 12:40:00,35.25,-770.10,35.25,-770.10,5370587.0,2003-07-02 12:40:00,35.27,-77.12,35.27,-77.12,0.3,0,49,0,1332
4,2003-07-01 21:03:00,2003-07-01 21:03:00,32.67,-81.60,32.75,-813.02,5370139.0,2003-07-01 21:03:00,32.68,-81.60,32.75,-81.53,6.0,2,251,0,1326
5,2003-07-01 22:02:00,2003-07-01 22:20:00,32.83,-810.15,32.93,-81.05,5370232.0,2003-07-01 22:02:00,32.83,-81.17,32.93,-81.05,10.0,1,49,0,507
6,2003-07-02 02:00:00,2003-07-02 02:05:00,33.60,-795.02,33.63,-795.02,5373890.0,2003-07-02 02:00:00,33.60,-79.85,33.63,-79.85,1.0,1,89,0,523
7,2003-07-02 18:00:00,2003-07-02 18:03:00,34.53,-784.05,34.53,-78.65,5373332.0,2003-07-02 18:00:00,34.53,-78.72,34.53,-78.65,0.8,1,17,0,525
8,2003-07-01 04:29:00,2003-07-01 04:35:00,32.18,-86.38,32.25,-86.38,5375712.0,2003-07-01 04:29:00,32.20,-86.38,32.25,-86.38,4.7,0,101,0,533
